In [2]:
import numpy as np 
import itertools as iter    
from collections import defaultdict
import statistics as st 

def generateCasinoResults(size):
    probabilities = np.array(((np.ones(6) / 6), (np.ones(6) / 10)))
    switchProbability = np.array((0.04, 0.05))
    probabilities[1][-1] = 0.5
    throws = []
    dices = []
    current_dice = 0
    for i in range(size):
        dices.append(current_dice)
        throws.append(np.random.choice(np.arange(1,7), p = probabilities[current_dice]))
        
        if np.random.rand() <= switchProbability[current_dice]:
            current_dice = 1 - current_dice
    
    return np.array(throws), np.array(dices)


In [39]:
data, dices = generateCasinoResults(10000)

In [40]:
def heuristicSolution(interval, minSixCount):
    res = np.zeros(data.size, dtype=int)
    for i in range(0, len(data), interval):
        test = data[i:i+interval]
        res[i:i+interval] = int((test == 6).sum() >= minSixCount)
    return res

In [41]:
heuristicResult = heuristicSolution(20, 7)

score = (heuristicResult == dices).mean()

print(score)

0.7721


In [42]:

def viterbi():
    probabilities = np.array(((np.ones(6) / 6), (np.ones(6) / 10)))
    switchProbability = np.array((0.04, 0.05))
    probabilities[1][-1] = 0.5
    a = np.array([
        [1-switchProbability[0], switchProbability[0]],
        [switchProbability[1], 1-switchProbability[1]]
    ])
    b = np.array([
        [probabilities[0], probabilities[0]],
        [probabilities[1], probabilities[1]]
    ])
    pi=np.array([1,0])

    alfa = np.zeros((2, data.size), dtype=np.float128)

    alfa[:, 0] = pi

    for t in range(1, data.size):
        for j in range(2):
            for i in range(2):
                alfa[j,t] += alfa[i, t-1] * a[i,j] * b[i, j, data[t-1]-1]
            alfa[j,t] *= 2

    beta = np.zeros((2, data.size), dtype=np.float128)
    beta[:, data.size-1] = 1

    for t in range(data.size-2, -1, -1):
        for j in range(2):
            for i in range(2):
                beta[j,t] += beta[i, t+1] * a[i, j] * b[i, j, data[t]-1]
            beta[j,t] *= 2
    

    gamma = alfa * beta / (alfa * beta).sum()

    return gamma.argmax(0)


In [43]:
resultViterbi = viterbi()
scoreViterbi = (resultViterbi == dices).mean()

print(scoreViterbi)

0.8482
